<a href="https://colab.research.google.com/github/lmcanavals/cs_topics/blob/main/0303_more_or_tools_happiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

     |████████████████████████████████| 14.4 MB 95 kB/s 


In [ ]:
from ortools.sat.python import cp_model

#First example

In [ ]:
#crear el modelo CSP
model = cp_model.CpModel()

In [ ]:
#variables y dominios
x = model.NewIntVar(0,2,'x')
y = model.NewIntVar(0,2,'y')
z = model.NewIntVar(0,2,'z')
#restricciones
model.Add(x != y)
model.Add(y == z + 1)

In [ ]:
#crear el solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  print('x = {:d}'.format(solver.Value(x)))
  print('y = {:d}'.format(solver.Value(y)))
  print('z = {:d}'.format(solver.Value(z)))

x = 0
y = 1
z = 0


#Cryptogram

##Brute-force

In [ ]:
def cryptogramBF():
  for s in range(1,10):
    for e in range(10):
      for n in range(10):
        for d in range(10):
          for m in range(1,10):
            for o in range(10):
              for r in range(10):
                for y in range(10):
                  l = [s,e,n,d,m,o,r,y]
                  c = set(l)
                  if len(l) == len(c) and 1000*(s + m) + 100*(e + o) + 10*(n + r) + (d + e) == 10000*m + 1000*o + 100*n + 10*e + y:
                    print(' ',s,e,n,d,'+')
                    print(' ',m,o,r,e)
                    print('---------')
                    print(m,o,n,e,y)
cryptogramBF()

  9 5 6 7 +
  1 0 8 5
---------
1 0 6 5 2


In [ ]:
#crear CSP
model = cp_model.CpModel()
#variables y dominios
s = model.NewIntVar(1,9,'s')
e = model.NewIntVar(0,9,'e')
n = model.NewIntVar(0,9,'n')
d = model.NewIntVar(0,9,'d')
m = model.NewIntVar(1,1,'m')
o = model.NewIntVar(0,9,'o')
r = model.NewIntVar(0,9,'r')
y = model.NewIntVar(0,9,'y')
v = [s,e,n,d,m,o,r,y]
#restricciones
model.Add(1000 * (s+m) + 100 * (e+o) + 10 * (n+r) + (d+e) == 10000*m+1000*o+100*n+10*e+y)
model.AddAllDifferent(v)

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  s,e,n,d,m,o,r,y = solver.Value(s),solver.Value(e),solver.Value(n),solver.Value(d),solver.Value(m),solver.Value(o),solver.Value(r),solver.Value(y)
  print(' ',s,e,n,d,'+')
  print(' ',m,o,r,e)
  print('---------')
  print(m,o,n,e,y)

  9 5 6 7 +
  1 0 8 5
---------
1 0 6 5 2


In [ ]:
print(solver.StatusName(4))

OPTIMAL


##Latin Square

In [ ]:
n = 9
#crear CSP
model = cp_model.CpModel()
#variables y dominios
grilla = []
for i in range(n):
  fila = []
  for j in range(n):
    fila += [model.NewIntVar(1,n,'x'+str(i)+str(j))]
  #fila = [model.NewIntVar(1,n,'x'+str(i)+str(j)) for j in range(n)]
  grilla += [fila]
#print(grilla)
#restricciones
##toda fila tiene valores distintos
for i in range(n):
  model.AddAllDifferent(grilla[i])
##toda columna tiene valores distintos
for j in range(n):
  #recuperar columna
  c = [grilla[i][j] for i in range(n)]
  #print(c)
  model.AddAllDifferent(c)

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
  for i in range(n):
    for j in range(n):
      print(solver.Value(grilla[i][j]),end=' ')
    print()

1 2 4 7 5 6 8 9 3 
6 4 9 8 7 5 3 1 2 
3 7 6 9 8 1 2 4 5 
4 8 2 1 9 3 7 5 6 
5 3 1 4 2 8 9 6 7 
7 1 8 5 3 9 6 2 4 
9 5 3 6 4 2 1 7 8 
8 9 7 2 6 4 5 3 1 
2 6 5 3 1 7 4 8 9 


#Magic Square

In [ ]:
n = 4
s = int((n**2)*(1+n**2)/(2*n))
print(s)
#crear CSP
model = cp_model.CpModel()
#variables y dominios
grilla = []
for i in range(n):
  fila = []
  for j in range(n):
    fila += [model.NewIntVar(1,n**2,'x'+str(i)+str(j))]
  grilla += [fila]
#print(grilla)
#restricciones
##recuperar todos los valores de la grilla en un solo arreglo
v = []
for i in range(n):
  v += grilla[i]
#print(v)
model.AddAllDifferent(v)
##todas las filas deben sumar el mismo valor
for i in range(n):
  model.Add(sum(grilla[i]) == s)
##todas las columnas deben sumar el mismo valor
for j in range(n):
  c = [grilla[i][j] for i in range(n)]
  model.Add(sum(c) == s)

34


In [ ]:
#one solution
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  for i in range(n):
    for j in range(n):
      print(solver.Value(grilla[i][j]),end=' ')
    print()

1 2 16 15 
12 5 9 8 
10 14 3 7 
11 13 6 4 


In [ ]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [ ]:
#all solutions
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(v)
status = solver.SearchForAllSolutions(model,solution_printer)
print(solution_printer.solution_count())

x00=16 x01=13 x02=2 x03=3 x10=9 x11=12 x12=7 x13=6 x20=8 x21=5 x22=10 x23=11 x30=1 x31=4 x32=15 x33=14 
x00=7 x01=13 x02=10 x03=4 x10=14 x11=1 x12=3 x13=16 x20=5 x21=11 x22=6 x23=12 x30=8 x31=9 x32=15 x33=2 
x00=3 x01=6 x02=10 x03=15 x10=16 x11=4 x12=9 x13=5 x20=1 x21=13 x22=8 x23=12 x30=14 x31=11 x32=7 x33=2 
x00=16 x01=8 x02=6 x03=4 x10=1 x11=15 x12=5 x13=13 x20=14 x21=2 x22=11 x23=7 x30=3 x31=9 x32=12 x33=10 
x00=2 x01=14 x02=15 x03=3 x10=11 x11=10 x12=7 x13=6 x20=5 x21=9 x22=8 x23=12 x30=16 x31=1 x32=4 x33=13 
x00=1 x01=13 x02=12 x03=8 x10=15 x11=6 x12=3 x13=10 x20=16 x21=4 x22=5 x23=9 x30=2 x31=11 x32=14 x33=7 
x00=2 x01=13 x02=16 x03=3 x10=7 x11=9 x12=12 x13=6 x20=14 x21=4 x22=1 x23=15 x30=11 x31=8 x32=5 x33=10 
x00=5 x01=12 x02=4 x03=13 x10=11 x11=7 x12=14 x13=2 x20=8 x21=9 x22=1 x23=16 x30=10 x31=6 x32=15 x33=3 
x00=10 x01=7 x02=11 x03=6 x10=14 x11=13 x12=3 x13=4 x20=1 x21=12 x22=5 x23=16 x30=9 x31=2 x32=15 x33=8 
x00=6 x01=11 x02=10 x03=7 x10=2 x11=1 x12=15 x13=16 x20=14 x21=1

#Winter school

In [ ]:
#crear CSP
model = cp_model.CpModel()
#variables y dominios
sesion = []
for i in range(11): #sesiones
  curso = []
  for d in range(2): #dias
    dias =[]
    for t in range(2): #turnos
      turno = []
      for s in range(3): #salones
        turno += [model.NewBoolVar('x'+str(i)+'_'+str(d)+'_'+str(t)+'_'+str(s))]
      dias += [turno]
    curso += [dias]
  sesion += [curso]
print(sesion)
#restricciones
##cada curso se dicta 1 vez
for i in range(11):
  l = []
  for d in range(2):
    for t in range(2):
      for s in range(3):
        l += [sesion[i][d][t][s]]
  #print(l)
  model.Add(sum(l) == 1)
##cada salon solo hasta 1 curso al mismo tiempo
for d in range(2):
  for t in range(2):
    for s in range(3):
      l = [sesion[i][d][t][s] for i in range(11)]
      model.Add(sum(l) <= 1)
##no se pueden hacer en simultaneo ciertas sesiones
def no_simultaneo(m, a, b):
  for d in range(2):
    for t in range(2):
      m.Add(sum(sesion[a][d][t]) + sum(sesion[b][d][t]) <= 1)

#A B C D E F G H I J K
#0 1 2 3 4 5 6 7 8 9 10

#A y J
no_simultaneo(model, 0, 9)
#J e I
no_simultaneo(model, 9, 8)
#C y F
no_simultaneo(model, 2, 5)
#F and G
no_simultaneo(model, 5, 6)
# D and H
no_simultaneo(model, 3, 7)
# B and D
no_simultaneo(model, 1, 3)
# K and E
no_simultaneo(model, 10, 4)
#B, I, H and G
no_simultaneo(model, 1, 8) #B,I
no_simultaneo(model, 1, 7) #B,H
no_simultaneo(model, 1, 6) #B,G
no_simultaneo(model, 8, 7) #I,H
no_simultaneo(model, 8, 6) #I,G
no_simultaneo(model, 7, 6) #H,G
#A, G and E
#B, H and K
#A, B, C and H
#D, F and J

#E before J
model.Add(sum(sesion[9][0][0]) == 0)
model.Add(sum(sesion[4][0][0]) >= sum(sesion[9][0][1]))
model.Add(sum(sesion[4][0][0]) + sum(sesion[4][0][1]) >= sum(sesion[9][1][0]))
model.Add(sum(sesion[4][1][0]) >= sum(sesion[9][1][1]))

[[[[x0_0_0_0(0..1), x0_0_0_1(0..1), x0_0_0_2(0..1)], [x0_0_1_0(0..1), x0_0_1_1(0..1), x0_0_1_2(0..1)]], [[x0_1_0_0(0..1), x0_1_0_1(0..1), x0_1_0_2(0..1)], [x0_1_1_0(0..1), x0_1_1_1(0..1), x0_1_1_2(0..1)]]], [[[x1_0_0_0(0..1), x1_0_0_1(0..1), x1_0_0_2(0..1)], [x1_0_1_0(0..1), x1_0_1_1(0..1), x1_0_1_2(0..1)]], [[x1_1_0_0(0..1), x1_1_0_1(0..1), x1_1_0_2(0..1)], [x1_1_1_0(0..1), x1_1_1_1(0..1), x1_1_1_2(0..1)]]], [[[x2_0_0_0(0..1), x2_0_0_1(0..1), x2_0_0_2(0..1)], [x2_0_1_0(0..1), x2_0_1_1(0..1), x2_0_1_2(0..1)]], [[x2_1_0_0(0..1), x2_1_0_1(0..1), x2_1_0_2(0..1)], [x2_1_1_0(0..1), x2_1_1_1(0..1), x2_1_1_2(0..1)]]], [[[x3_0_0_0(0..1), x3_0_0_1(0..1), x3_0_0_2(0..1)], [x3_0_1_0(0..1), x3_0_1_1(0..1), x3_0_1_2(0..1)]], [[x3_1_0_0(0..1), x3_1_0_1(0..1), x3_1_0_2(0..1)], [x3_1_1_0(0..1), x3_1_1_1(0..1), x3_1_1_2(0..1)]]], [[[x4_0_0_0(0..1), x4_0_0_1(0..1), x4_0_0_2(0..1)], [x4_0_1_0(0..1), x4_0_1_1(0..1), x4_0_1_2(0..1)]], [[x4_1_0_0(0..1), x4_1_0_1(0..1), x4_1_0_2(0..1)], [x4_1_1_0(0..1), x4_1

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  for i in range(11):
    print(i,end='\t')
    for d in range(2):
      for t in range(2):
        for s in range(3):
          print(solver.Value(sesion[i][d][t][s]),end=' ')
        print("",end='- ')
      print("",end="\t| ")
    print()

  print("-----------")
  for d in range(2):
    print('d'+str(d+1),'\t',end=' ')
    for t in range(2):
      for s in range(2):
        for i in range(11):
          if solver.Value(sesion[i][d][t][s]) == 1:
            print("curso"+str(i), end=' ')
      print('',end='|')
    print()

0	0 0 1 - 0 0 0 - 	| 0 0 0 - 0 0 0 - 	| 
1	1 0 0 - 0 0 0 - 	| 0 0 0 - 0 0 0 - 	| 
2	0 0 0 - 0 0 0 - 	| 0 0 0 - 0 1 0 - 	| 
3	0 0 0 - 0 1 0 - 	| 0 0 0 - 0 0 0 - 	| 
4	0 0 0 - 1 0 0 - 	| 0 0 0 - 0 0 0 - 	| 
5	0 1 0 - 0 0 0 - 	| 0 0 0 - 0 0 0 - 	| 
6	0 0 0 - 0 0 0 - 	| 0 0 0 - 1 0 0 - 	| 
7	0 0 0 - 0 0 0 - 	| 1 0 0 - 0 0 0 - 	| 
8	0 0 0 - 0 0 1 - 	| 0 0 0 - 0 0 0 - 	| 
9	0 0 0 - 0 0 0 - 	| 0 0 0 - 0 0 1 - 	| 
10	0 0 0 - 0 0 0 - 	| 0 0 1 - 0 0 0 - 	| 
-----------
d1 	 curso1 curso5 |curso4 curso3 |
d2 	 curso7 |curso6 curso2 |
